In [ ]:
'''
@ author : haijun xiong
@ date   : 25/Sep/2020

'''
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
def MSE_loss(y_hat, y):
    return 1/2 * (y_hat - y)**2
class LinearModel():
    def __init__(self, train_data, epoch, lr):
        x = np.concatenate((train_data[:, :-1], np.ones((train_data.shape[0], 1))), axis=1)
        y = train_data[:, -1]
        self.x = x
        self.y = y.reshape(-1,1)
        self.w = np.zeros((x.shape[1], 1))
        self.lr = lr
        self.epochs = epoch
    def train(self):
        for epoch in range(self.epochs):
            y_hat = self.x @ self.w
            loss = MSE_loss(y_hat, self.y)
            w_grad = 2 * self.x.T @ (y_hat - self.y)
            self.w -= self.lr * w_grad.reshape(-1,1)
            if epoch % 10 == 0:
                print('epoch:{}, loss:{}'.format(epoch, np.sum(loss)))
            if epoch % 100 == 0:
                self.lr *= 0.1
    def test(self, test_x):
        y_hat = test_x @ self.w
        return y_hat

    def evaluate(self, test_data):
        test_x = np.concatenate((test_data[:, :-1], np.ones((test_data.shape[0], 1))), axis=1)
        test_y = test_data[:, -1].reshape(-1,1)
        y_hat = self.test(test_x)
        y_hat[y_hat < 0] = - 1
        y_hat[y_hat > 0] = 1
        acc = np.sum(y_hat == test_y) / len(test_y)
        return acc

In [ ]:
def get_cov(d):
    return np.identity(d)

def get_mean(d):
    mean = np.zeros(d)
    mean[0] = 1
    return mean

In [ ]:
def get_dataset(mu, cov, nums):
    scale = 0.8
    data1 = np.random.multivariate_normal(mean=mean, cov=cov, size=nums)
    data2 = np.random.multivariate_normal(mean=-1*mu, cov=cov, size=nums)
    data1 = np.concatenate((data1, -1 * np.ones((nums, 1))), axis=1)
    data2 = np.concatenate((data2, np.ones((nums, 1))), axis=1)
    train_data1, val_data1 = split_dataset(data1, scale) 
    train_data2, val_data2 = split_dataset(data2, scale)
    train_data = np.vstack((train_data1, train_data2)).astype(np.float32)
    val_data = np.vstack((val_data1, val_data2)).astype(np.float32)
    return train_data, val_data

def split_dataset(dataset, scale):
    num_data = dataset.shape[0]
    train_dataset = []
    valid_dataset = []
    train_indice = random.sample(range(num_data), int(num_data * scale))
    for index in range(num_data):
        if index in train_indice:
            train_dataset.append(dataset[index])
        else:
            valid_dataset.append(dataset[index])
    return np.array(train_dataset), np.array(valid_dataset)

In [ ]:
def show_clf(data, w, nums=500):
    x = data[:, :-1]
    y = data[:, -1]
    xx = np.linspace(-4, 5, nums)
    a = w[0]
    b = w[1]
    c = w[2]
    yy = -(a/b)*xx - (c/b)
    plt.figure()
    plt.scatter(x[:,0], x[:,1], c=y, alpha=0.2)
    plt.plot(xx, yy, c='r')
    plt.show()

In [ ]:
np.random.seed(1)
epoch = 400
nums = 250
d = 2
mean = np.array([1, -1])
cov = get_cov(d)
train_data, test_data = get_dataset(mean, cov, nums)

In [ ]:
model = LinearModel(train_data, epoch, lr=1e-3)

In [ ]:
model.train()

In [ ]:
print(model.w)
show_clf(train_data, model.w, 2 * nums)     # plot the figure

In [ ]:
show_clf(test_data, model.w, 2 * nums) 

In [ ]:
acc = model.evaluate(test_data)

In [ ]:
d = 500
mean = get_mean(d)
cov = get_cov(d)
train_data, test_data = get_dataset(mean, cov, nums)
model = LinearModel(train_data, epoch * 10, lr=1e-4)
model.train()

In [ ]:
acc = model.evaluate(test_data)